## pixelCNN -> masked conv layer, residual block

In [1]:
from tensorflow.keras import layers
import numpy as np

# masked conv layer

class Maskedconvlayer(layers.Layer):
    def __init__(self, mask_type, **kwargs):
        super(Maskedconvlayer, self).__init__()
        self.mask_type = mask_type
        self.conv = layers.Conv2D(**kwargs)
    
    def build(self, input_shape):
        self.conv_build(input_shape)
        kernel_shape = self.conv.kernel.get_shape()
        self.mask = np.zeros(shape = kernel_shape)
        self.mask[: kernel_shape[0] // 2, ...] = 1.0
        self.mask[kernel_shape[0] // 2, :kernel_shape[1] // 2, ...] = 1.0
        if self.mask_type == 'B':
            self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ... ] = 1.0
        
    def call(self, inputs):
        self.conv.kernel.assign(self.conv.kernel * self.mask)
        return self.conv(inputs)


In [ ]:
# Residual block 


class Residualblock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(Residualblock, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(filters = filters // 2, kernel_size = 1, activation = 'relu')
        self.pixel_conv = layers.MaskedConv2D(mask_type = 'B', filters = filters // 2, kernel_size = 3, activation = 'relu', padding = 'same')
        self.conv2 = layers.Conv2D(filters = filters, kernel_size = 1, activation = 'relu')
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pixel_conv(x)
        x = self.conv2(x)
        return layers.add([inputs, x])
    
